In [12]:
import pandas as pd
import numpy as np
import re
import os

df = pd.read_json('../Data/Serie_A_2013_games.json')

df_squad = pd.read_json('../Data/Serie_A_2013_squads.json').T

df_transposto = df.transpose()

df_transposto.head(20)

,Home,Away,Result,Players,Goals,Changes,Yellow cards,Red cards
1,Vasco da Gama / RJ,Portuguesa / SP,1 X 0,[[1Michel Alves Michel Aluizio da Cr ... T(g)P...,[02:00 2T11NRCarlos Vicente Tenorio Medina Vas...,[17:00 2TVasco da Gama 19 - Edmilson dos Santo...,[],[]
2,Fluminense / RJ,Athletico Paranaense / PR,2 X 1,"[[1Ricardo Ricardo Ferreira Berna T(g)P135462,...",[27:00 1T3NRManoel Messias Silva Carvalho Atlé...,[45:00 INTFluminense 38 - Eduardo Antonio Mach...,[],[]
3,Corinthians / SP,Botafogo / RJ,1 X 1,"[[4Gil Carlos Gilberto do N ... T(g)P178854, C...","[25:00 1T20NRRafael Marques Mariano Botafogo, ...",[45:00 INTCorinthians 10 - Douglas dos Santos ...,[],[]
4,Vitória / BA,Internacional / RS,2 X 2,[[1Junior Wilson Rodrigues de ... T(g)P155508...,[02:00 1T7NRMaximiliano Daniel Biancucchi Vitó...,[45:00 INTVitória 18 - Giancarlo Lopes Rodrigu...,[],[]
5,Grêmio / RS,Náutico / PE,2 X 0,"[[1Dida Nelson de Jesus Silva T(g)P117336, Grê...",[15:00 1T10NRJose Roberto da Silva Junior Grêm...,[26:00 2TNáutico 17 - Adeilson Pereira de Mell...,[],[]
6,Ponte Preta / SP,São Paulo / SP,0 X 2,[[1Edson Bastos Edson Bastos Barreto T(g)P1409...,[08:00 1T3NRLucimar da Silva Ferreira São Paul...,[16:00 2TSão Paulo 13 - Jonathan Doin 22 - Sil...,[],[]
7,Criciúma / SC,Bahia / BA,3 X 1,"[[1Bruno Bruno Brigido de Oli ... T(g)P305735,...","[45:00 1T3NRMatheus Ferraz Pereira Criciuma, 0...",[02:00 2TBahia 12 - Omar Constante Reis Santo ...,[],[]
8,Cruzeiro / MG,Goiás / GO,5 X 0,"[[1Fabio Fabio Deivson Lopes ... T(g)P129292,...","[06:00 1T10NRDiego de Souza Andrade Cruzeiro, ...",[45:00 INTGoiás 15 - Ramon Rodrigo de Freitas ...,[],[]
9,Coritiba / PR,Atlético Mineiro / MG,2 X 1,[[1Vanderlei Vanderlei Farias da ... T(g)P177...,"[04:00 2T9NRDiego Tardelli Martins Atlético, 0...",[20:00 2TAtlético 77 - Sergio Antonio Borges J...,[],[]
10,Santos / SP,Flamengo / RJ,0 X 0,"[[1Rafael Rafael Cabral Barbosa T(g)P292253, S...",[],[45:00 INTFlamengo/RJ 19 - Marcelo Moreno Mart...,[36:00 1T9Jose Henrique da Silva Dourado Santo...,[]


In [67]:
# Selecionar linhas onde 'Palmeiras / SP' aparece em 'Home' ou 'Away'
#df_transposto.loc[(df_transposto['Home'] == 'Palmeiras / SP') | (df_transposto['Away'] == 'Palmeiras / SP')]


In [105]:
df_squad.head(9)

,0,1,2,3,4,5,6,7,8
1,"{'Home': {'Squad': ['146379', '186825', '43089...","{'Home': {'Squad': ['146379', '186825', '43089...","{'Home': {'Squad': ['146379', '186825', '43089...","{'Home': {'Squad': ['146379', '186825', '43089...","{'Home': {'Squad': ['146379', '186825', '43089...","{'Home': {'Squad': ['146379', '186825', '43089...",NaN,NaN,NaN
2,"{'Home': {'Squad': ['135462', '167958', '14722...","{'Home': {'Squad': ['135462', '167958', '14722...","{'Home': {'Squad': ['135462', '167958', '14722...","{'Home': {'Squad': ['135462', '167958', '14722...","{'Home': {'Squad': ['135462', '167958', '14722...","{'Home': {'Squad': ['135462', '167958', '14722...",NaN,NaN,NaN
3,"{'Home': {'Squad': ['178854', '168387', '16224...","{'Home': {'Squad': ['178854', '168387', '16224...","{'Home': {'Squad': ['178854', '168387', '16224...","{'Home': {'Squad': ['178854', '168387', '16224...","{'Home': {'Squad': ['178854', '168387', '16224...","{'Home': {'Squad': ['178854', '168387', '16224...",NaN,NaN,NaN
4,"{'Home': {'Squad': ['155508', '183977', '17715...","{'Home': {'Squad': ['155508', '183977', '17715...","{'Home': {'Squad': ['155508', '183977', '17715...","{'Home': {'Squad': ['155508', '183977', '17715...","{'Home': {'Squad': ['155508', '183977', '17715...","{'Home': {'Squad': ['155508', '183977', '17715...",NaN,NaN,NaN
5,"{'Home': {'Squad': ['117336', '165065', '17241...","{'Home': {'Squad': ['117336', '165065', '17241...","{'Home': {'Squad': ['117336', '165065', '17241...","{'Home': {'Squad': ['117336', '165065', '17241...","{'Home': {'Squad': ['117336', '165065', '17241...",NaN,NaN,NaN,NaN
6,"{'Home': {'Squad': ['140911', '185679', '29741...","{'Home': {'Squad': ['140911', '185679', '29741...","{'Home': {'Squad': ['140911', '185679', '29741...","{'Home': {'Squad': ['140911', '185679', '29741...","{'Home': {'Squad': ['140911', '185679', '29741...","{'Home': {'Squad': ['140911', '185679', '29741...",NaN,NaN,NaN
7,"{'Home': {'Squad': ['305735', '186144', '17547...","{'Home': {'Squad': ['305735', '186144', '17547...","{'Home': {'Squad': ['305735', '186144', '17547...","{'Home': {'Squad': ['305735', '186144', '17547...","{'Home': {'Squad': ['305735', '186144', '17547...","{'Home': {'Squad': ['305735', '186144', '17547...",NaN,NaN,NaN
8,"{'Home': {'Squad': ['129292', '138932', '15728...","{'Home': {'Squad': ['129292', '138932', '15728...","{'Home': {'Squad': ['129292', '138932', '15728...","{'Home': {'Squad': ['129292', '138932', '15728...","{'Home': {'Squad': ['129292', '138932', '15728...",NaN,NaN,NaN,NaN
9,"{'Home': {'Squad': ['177770', '170272', '18955...","{'Home': {'Squad': ['177770', '170272', '18955...","{'Home': {'Squad': ['177770', '170272', '18955...","{'Home': {'Squad': ['177770', '170272', '18955...","{'Home': {'Squad': ['177770', '170272', '18955...","{'Home': {'Squad': ['177770', '170272', '18955...","{'Home': {'Squad': ['177770', '170272', '18955...",NaN,NaN


Teste inicial utilizando um jogo do brasileirão Série A 2014. Pegamos o primeiro jogo do Fluminense x Figueirense para criarmos a formatação das colunas e extração de informações da súmula.

In [13]:
players_teams = df_transposto.iloc[6, 3]
team_changes = df_transposto.iloc[6, 5]

start_players = df_squad.iloc[6,:]

# Criando DataFrame dos jogadores
new_df_players = pd.DataFrame(players_teams, columns=['player', 'team'])

def extract_id(player_string):
    # Ajuste para garantir que captura corretamente todas as variantes de ID
    match = re.search(r'T\(g\)?P(\d+)|RP(\d+)|TP(\d+)', player_string, re.IGNORECASE)
    if match:
        return next((m for m in match.groups() if m), None)
    return None

# Limpeza dos nomes dos jogadores
def clean_player_name(player):
    clean_name = re.sub(r'\s+T.*|\s+TP.*|\s+RP.*|\s+T(g).*', '', player)  # Remove texto após 'T' ou 'TP'
    return clean_name

new_df_players['player_id'] = new_df_players['player'].apply(extract_id)
new_df_players['player_name'] = new_df_players['player'].apply(clean_player_name)


def parse_team_changes(changes):
    pattern = re.compile(r'(\d{2}:\d{2}) (INT|\d+T)([\w\s]+/\w+) (\d+) - [^\d]+ (\d+) - [^\d]+')
    parsed_data = []
    for change in changes:

        match = pattern.search(change)
        if match:
            time, half, team, player_out_number, player_in_number = match.groups()
            team = team.strip()  # Limpeza para remover espaços extras
            parsed_data.append((time, half, team, player_out_number, player_in_number))
    return parsed_data


parsed_changes = parse_team_changes(team_changes)
print(team_changes)


new_df_players['Minutes Played'] = 90
new_df_players['Minute Entered'] = 0  # Todos os jogadores começam com entrada no minuto 0
new_df_players['Minute Exited'] = 90  # Todos os jogadores começam com saída no minuto 90

def verifica_substring(substring, string):
    partes = substring.split()
    return all(parte in string for parte in partes)


def tratar_excecoes_nomes_times(team):

    # Dicionário de exceções: chave é o nome parcial, valor é o nome completo
    excecoes = {
        "Atlético/PR": "Athletico Paranaense/PR",
        "Atlético/MG": "Atlético Mineiro/MG"
        # Adicione outras exceções aqui conforme necessário
    }
    
    # Verificar se o time está nas exceções e substituí-lo
    return excecoes.get(team, team)


for time, half, team, player_out_number, player_in_number in parsed_changes:

    team = tratar_excecoes_nomes_times(team)
    minute = int(time.split(':')[0])
    team = team.replace('/', ' / ')  # Formatando o nome do time corretamente
    
    if '2T' in half:
        minute_entered = 45 + minute
        minute_exited = 45 + minute
    else:
        minute_entered = minute
        minute_exited = minute

    # Atualizar o jogador que entrou
    mask_in = (new_df_players['player_name'].apply(lambda x: re.match(r'^' + player_in_number + r'\D', x) is not None) & 
               new_df_players['team'].apply(lambda x: verifica_substring(team, x)))
    new_df_players.loc[mask_in, 'Minute Entered'] = minute_entered
    new_df_players.loc[mask_in, 'Minute Exited'] = 90  # O jogador que entra fica até o final do jogo

    # Atualizar o jogador que saiu
    mask_out = (new_df_players['player_name'].apply(lambda x: re.match(r'^' + player_out_number + r'\D', x) is not None) & 
                new_df_players['team'].apply(lambda x: verifica_substring(team, x)))
    new_df_players.loc[mask_out, 'Minute Exited'] = minute_exited

new_df_players['Minutes Played'] = new_df_players['Minute Exited'] - new_df_players['Minute Entered']


#setando Home ou Away
home_team = df_transposto.iloc[6, 0]
new_df_players
new_df_players['status'] = np.where(new_df_players['team'] == home_team, 'Home', 'Away')

new_df_players = new_df_players.drop(columns='player')
# Visualizando o novo DataFrame
new_df_players

['02:00 2TBahia 12 - Omar Constante Reis Santo ... 1 - Marcelo Lomba do Nascimen ...', '09:00 2TBahia 16 - Diones Coelho da Costa 11 - Italo Melo Oliveira', '25:00 2TCriciuma 17 - Vinicius Silva Soares 11 - Fabio Souza dos Santos', '33:00 2TCriciuma 14 - Gilson Gomes do Nasciment ... 9 - Marcel Augusto Ortolan', '36:00 2TBahia 17 - Fredua Koranteng Adu 8 - Ryder Matos Santos Pinto']


,team,player_id,player_name,Minutes Played,Minute Entered,Minute Exited,status
0,Criciúma / SC,305735,1Bruno Bruno Brigido de Oli ...,90,0,90,Home
1,Criciúma / SC,186144,2Bruno Bruno Renan,90,0,90,Home
2,Criciúma / SC,175475,3Matheus Matheus Ferraz Pereira,90,0,90,Home
3,Criciúma / SC,182550,4Ewerton Ewerton Ribeiro Pascoa,90,0,90,Home
4,Criciúma / SC,182625,5Tiago Dutra,90,0,90,Home
5,Criciúma / SC,183840,6Marlon Marlon Farias Castel ...,90,0,90,Home
6,Criciúma / SC,176375,7Lins Lins Lima de Brito,90,0,90,Home
7,Criciúma / SC,181803,8Elton Elton Junior Melo Ataide,90,0,90,Home
8,Criciúma / SC,139659,9Marcel Marcel Augusto Ortolan,90,0,90,Home
9,Criciúma / SC,164019,10Joao Vitor Joao Vitor Lima Gomes,90,0,90,Home


In [22]:
import unicodedata

def tratar_excecoes_nomes_times_2(team):
    # Dicionário de exceções: chave é o nome parcial, valor é o nome completo
    excecoes = {
        "Athletico Paranaense/PR": "Atlético/PR",
        "Atlético Mineiro/MG": "Atlético/MG"
        # Adicione outras exceções aqui conforme necessário
    }
    
    # Verificar se o time está nas exceções e substituí-lo
    return excecoes.get(team, team)

def remove_accents(text):
    # Normalize the text to NFKD form and remove accents
    return ''.join(c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c))


def parse_goals(goals, home_team):
    pattern = re.compile(r'(\+?\d+)(:00)? (\d+T)([^\s]+)\s+([^\d]+?)\s+(.*)')

    
    parsed_goals = []
    for goal in goals:

        match = pattern.search(goal)


        if match:
            if '+' in (match.group(1)):
                minute = 45
            else:
                minute = int(match.group(1))

            half = match.group(3)

            scorer_info = match.group(4)

            player_name = match.group(5).strip()

            time_casa = df_transposto.iloc[6, 0]
            time_visitante = df_transposto.iloc[6, 1]
            print(match.group(6))
            print(time_casa)


            time_casa = time_casa.replace(' / ', '/')
            time_visitante = time_visitante.replace(' / ', '/')

            time_casa_except = tratar_excecoes_nomes_times_2(time_casa)

            time_casa_normalized = remove_accents(time_casa)
            time_casa_except_normalized = remove_accents(time_casa_except)

            if time_casa in  match.group(6) or time_casa_except in match.group(6) or time_casa_normalized in match.group(6) or time_casa_except_normalized in match.group(6):
                team = time_casa
            else: 
                team = time_visitante




            team = tratar_excecoes_nomes_times(team)
            team = team.replace('/', ' / ')
            # Convertendo para minuto total do jogo
            if '2T' in half and minute != 45:  # Ajusta para o tempo total no caso do segundo tempo
                minute += 45
            
            # Verificar se é um gol contra (CT)
            if 'CT' in scorer_info:
                if team == home_team:
                    team_status = 'Away'
                else:
                    team_status = 'Home'
            else:

                if team == home_team:
                    team_status = 'Home'
                else:
                    team_status = 'Away'
            
            parsed_goals.append((minute, team_status))
    return parsed_goals

home_team = df_transposto.iloc[6, 0]
parsed_goals = parse_goals(df_transposto.iloc[6,4], home_team)

parsed_goals

Pereira Criciuma
Criciúma / SC
de Brito Criciuma
Criciúma / SC
da Costa Bahia
Criciúma / SC
Lima Gomes Criciuma
Criciúma / SC


[(45, 'Away'), (46, 'Away'), (72, 'Away'), (80, 'Away')]

In [15]:

new_df_players['Goals For'] = 0

new_df_players['Goals Against'] = 0

for minute, team in parsed_goals:
    # Jogadores em campo no momento do gol
    mask = (new_df_players['Minute Entered'] <= minute) & (new_df_players['Minute Exited'] >= minute)
    # Incrementar gols a favor
    mask_for = mask & (new_df_players['status'] == team)
    new_df_players.loc[mask_for, 'Goals For'] += 1
    
    # Incrementar gols contra
    mask_against = mask & (new_df_players['status'] != team)
    new_df_players.loc[mask_against, 'Goals Against'] += 1

# Exibindo o DataFrame final para verificar os resultados
new_df_players

,team,player_id,player_name,Minutes Played,Minute Entered,Minute Exited,status,Goals For,Goals Against
0,Criciúma / SC,305735,1Bruno Bruno Brigido de Oli ...,90,0,90,Home,0,4
1,Criciúma / SC,186144,2Bruno Bruno Renan,90,0,90,Home,0,4
2,Criciúma / SC,175475,3Matheus Matheus Ferraz Pereira,90,0,90,Home,0,4
3,Criciúma / SC,182550,4Ewerton Ewerton Ribeiro Pascoa,90,0,90,Home,0,4
4,Criciúma / SC,182625,5Tiago Dutra,90,0,90,Home,0,4
5,Criciúma / SC,183840,6Marlon Marlon Farias Castel ...,90,0,90,Home,0,4
6,Criciúma / SC,176375,7Lins Lins Lima de Brito,90,0,90,Home,0,4
7,Criciúma / SC,181803,8Elton Elton Junior Melo Ataide,90,0,90,Home,0,4
8,Criciúma / SC,139659,9Marcel Marcel Augusto Ortolan,90,0,90,Home,0,4
9,Criciúma / SC,164019,10Joao Vitor Joao Vitor Lima Gomes,90,0,90,Home,0,4


In [74]:

def collect_unique_ids(data_folder, file_names, years):
    unique_ids = set()

    def process_dataframe(df):
        for column in df.columns:
            for cell in df[column]:
                if isinstance(cell, dict):  
                    for key in ['Home', 'Away']:
                        team_info = cell.get(key, {})
                        # Adicionar os IDs do 'Squad'
                        squad_ids = team_info.get('Squad', [])
                        unique_ids.update(squad_ids)


    for file_name in file_names:
        for year in years:
            file_path = os.path.join(data_folder, f'{file_name}_{year}_squads.json')
            if os.path.exists(file_path):
                df = pd.read_json(file_path)
                df_ = df.iloc[:,7]
                df = pd.DataFrame(df_)
                process_dataframe(df)
            else:
                print(f'Arquivo não encontrado: {file_path}')

    return list(unique_ids)

data_folder = '../Data/'
file_names = ['Serie_A']
years = range(2017, 2018)

unique_player_ids = collect_unique_ids(data_folder, file_names, years)

print(f'Total de IDs únicos coletados: {len(unique_player_ids)}')

Total de IDs únicos coletados: 28


In [75]:

df_players_final = new_df_players[new_df_players['player_id'].isin(unique_player_ids)]
df_players_final

,team,player_id,player_name,Minutes Played,Minute Entered,Minute Exited,status,Goals For,Goals Against
